In [41]:
from time import sleep

import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

In [5]:
torch.cuda.is_available()

False

# Env

In [3]:
import gymnasium as gym
import gym_pusht
from time import sleep

env = gym.make("gym_pusht/PushT-v0", render_mode="human")
observation, info = env.reset()

for _ in range(1000):
    sleep(0.2)
    action = env.action_space.sample()
    observation, reward, terminated, truncated, info = env.step(action)
    image = env.render()

    if terminated or truncated:
        observation, info = env.reset()

env.close()

KeyboardInterrupt: 

Implement the deepq Learning to compete the task

# Part 1: Numerical PushT state

## Observation Space

If obs_type is set to state, the observation space is a 5-dimensional vector representing the state of the environment: [agent_x, agent_y, block_x, block_y, block_angle]. The values are in the range [0, 512] for the agent and block positions and [0, 2*pi] for the block angle.

If obs_type is set to environment_state_agent_pos the observation space is a dictionary with: - environment_state: 16-dimensional vector representing the keypoint locations of the T (in [x0, y0, x1, y1, ...] format). The values are in the range [0, 512]. - agent_pos: A 2-dimensional vector representing the position of the robot end-effector.

If obs_type is set to pixels, the observation space is a 96x96 RGB image of the environment.

In [7]:
env.observation_space

Box(0.0, [512.         512.         512.         512.           6.28318531], (5,), float64)

In [8]:
from torch import nn
import torch.nn.functional as F


class Policy(nn.Module):
    def __init__(self):
        super(Policy, self).__init__()
        self.fc1 = nn.Linear(env.observation_space.shape[0], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

In [9]:
model = Policy()

In [11]:
input = torch.rand((4, 5))

input

tensor([[0.2108, 0.9965, 0.3429, 0.9719, 0.9332],
        [0.4103, 0.7770, 0.1434, 0.5347, 0.0748],
        [0.1812, 0.4294, 0.4721, 0.0404, 0.3892],
        [0.4568, 0.0204, 0.9001, 0.9913, 0.1264]])

In [19]:
output = model(input)


print(output.shape)
output.detach().numpy()

torch.Size([4, 2])


array([[-0.13495661, -0.06345759],
       [-0.1011354 , -0.05930006],
       [-0.07766321, -0.01074107],
       [-0.15807617, -0.0326845 ]], dtype=float32)

In [ ]:
def train_policy(policy_model, critic_model, input, optimizer):
    optimizer.zero_grad()

    input = input.to(device)

    output = policy_model(input)

    critic_input = torch.cat((input, output), dim=1)

    score = critic_model(critic_input)

    loss = -score

    loss.backward()
    optimizer.step()





In [38]:
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(env.observation_space.shape[0] + env.action_space.shape[0], 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))

        return x

In [39]:
model = Critic()
input = torch.rand((4, 7))

input

tensor([[0.9377, 0.4040, 0.7157, 0.2446, 0.1419, 0.2018, 0.9670],
        [0.9992, 0.8623, 0.5948, 0.7328, 0.3309, 0.3954, 0.7757],
        [0.6086, 0.6243, 0.2879, 0.9788, 0.8896, 0.9508, 0.4953],
        [0.3340, 0.2489, 0.5371, 0.2042, 0.8950, 0.6058, 0.4692]])

In [40]:
output = model(input)


print(output.shape)
output.detach().numpy()

torch.Size([4, 1])


array([[0.        ],
       [0.00227321],
       [0.        ],
       [0.00089526]], dtype=float32)

In [ ]:
def train_critic(model, input, target, optimizer):
    optimizer.zero_grad()
    criterion = nn.MSELoss()

    model.train()

    input = input.to(device)

    output = model(input)

    loss = criterion(output, target)

    loss.backward()
    optimizer.step()


In [ ]:
import random

class Memory:
    def __init__(self, batch_size = 128):
        self.items = []
        self.batch_size = batch_size

    def add(self, item):
        self.items.append(item)

    def sample(self):
        return random.sample(self.items, self.batch_size)

In [24]:
def train(model, memory, episodes = 10000, max_steps = 1000):
    env = gym.make("gym_pusht/PushT-v0", obs_type="state", render_mode="rgb_array")

    for ep in range(episodes):

        state, info = env.reset()

        for t in range(max_steps):

            state = torch.stack([torch.from_numpy(state).float()])

            action = model.forward(state)[0].detach().numpy()

            new_state, reward, terminated, truncated, info = env.step(action)

            sarsa = (state, action, reward, new_state)

            memory.add(sarsa)


            train_set = memory.sample()



            if terminated or truncated:
                observation, info = env.reset()


train(model)


KeyboardInterrupt: 